## MNIST Dataset 분류기 with ConvNet

* 한림대학교 딥러닝이해및활용 (710231)에 사용된 코드입니다. 

### 설정하기

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

데이터 받아오기

In [2]:
mnist = input_data.read_data_sets('./MNIST_data/', one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


Hyper parameters

In [3]:
learning_rate = 0.001
batch_size = 50
training_epochs = 1000#10000 # 20000번 정도 #전체 데이터 학습 횟수

### 필요 함수 정의

 초기화

In [4]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

Convolution and Pooling

In [5]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [6]:
X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, 10])

In [7]:
# reshape x to a 4d tensor
x_image = tf.reshape(X, [-1,28,28,1])

### Classifier Setup

<img src='./images/convnet1.png' width = '500px' />

#### First Convolutional Layer
*  The convolution will compute 32 features for each 5x5 patch.

In [8]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# We then convolve x_image with the weight tensor, add the bias, apply the ReLU function, and finally max pool.
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

# The max_pool_2x2 method will reduce the image size to 14x14.
h_pool1 = max_pool_2x2(h_conv1)

#### Second Conv Layer
* The second layer will have 64 features for each 5x5 patch.

In [9]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2) ## the image size will be reduced to 7x7

#### Densely Connected Layer (Fully connected layer)

In [10]:
n_neurons = 1024

본 예제에서는 이미지의  크기가  Convolution layer들을 거쳐오며 7x7로 줄어 들었음. 이때 이미지의 채널은 64개. 이걸 1024 개의 1차원 neuron에 연결할수 있도록 reshape 함.
reshape은다음과 같이 호출
```
tf.reshape(h_pool2, [-1, 7*7*64])
```

In [11]:
W_fc1 = weight_variable([7 * 7 * 64, n_neurons])
b_fc1 = bias_variable([n_neurons])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#### 마지막 Readout Layer
* 최종 10개로 연결 !

In [12]:
W_fc2 = weight_variable([n_neurons, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2

### Train and Evaluate the Model

In [13]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=y_conv))
cost = cross_entropy
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) #train_step
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(Y,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [14]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

ResourceExhaustedError: OOM when allocating tensor of shape [3136,1024] and type float
	 [[Node: Variable_4/Adam/Initializer/zeros = Const[_class=["loc:@Variable_4"], dtype=DT_FLOAT, value=Tensor<type: float shape: [3136,1024] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'Variable_4/Adam/Initializer/zeros', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hyunjoo/.venv/py3tf/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/hyunjoo/.venv/py3tf/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/hyunjoo/.venv/py3tf/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/hyunjoo/.venv/py3tf/l

Cost 를 기록하기 위한 empty list

In [ ]:
vcost=[]

In [ ]:
for i in range(training_epochs):
    batch = mnist.train.next_batch(batch_size)
    if i%100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={X:batch[0], Y: batch[1]})
        print("Epoch {}, training accuracy {}".format(i, train_accuracy))

    c, _ = sess.run([cost, optimizer], feed_dict={X: batch[0], Y: batch[1]})
    vcost.append(c)

In [ ]:
print(mnist.train.images.shape)
print(mnist.train.labels.shape)


In [ ]:
acc1 = sess.run(accuracy, feed_dict={X: mnist.train.images[:1000], Y: mnist.train.labels[:1000]})
print("Train accuracy :{}".format(acc1))

In [ ]:
acc1 = sess.run(accuracy, feed_dict={X: mnist.test.images[:1000], Y: mnist.test.labels[:1000]})
print("Test accuracy :{}".format(acc1))

Cost graph

In [ ]:
plt.plot(vcost)
plt.xlabel('Epoch')
plt.ylabel('Cost')
plt.show()

In [ ]:
sess.close()